In [1]:
import torch
import torchaudio
from IPython.display import Audio
from modules.wavlm import WavLM
from modules.hifi_gan import HiFiGAN
from modules.discrete_ssl import DiscreteSSL

/home/bltang/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def toks_to_sig(toks, num_clusters = 1000, ssl_layers = [1, 3, 7, 12, 18, 23]):
    # toks: [B, N, K]
    all_layer_ids = [1, 3, 7, 12, 18, 23]
    offsets = torch.arange(
            0,
            len(all_layer_ids) * num_clusters,
            num_clusters,
        )
    offset_idxes = [all_layer_ids.index(x) for x in ssl_layers]
    offsets = offsets[offset_idxes]
    toks = toks + offsets + 1
    return toks

In [3]:
## model
wavlm = WavLM("/home/bltang/work/benchmarks/models/wavlm-large")
hifi_gan = HiFiGAN("/home/bltang/work/benchmarks/models/hifigan-wavlm-l1-3-7-12-18-23-k1000-LibriTTS")
discrete_ssl = DiscreteSSL(wavlm, "LibriSpeech-100-360-500","/home/bltang/work/benchmarks/models/SSL_Quantization" )

Some weights of the model checkpoint at /home/bltang/work/benchmarks/models/wavlm-large were not used when initializing WavLMModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMModel were not initialized from the model checkpoint at /home/bltang/work/benchmarks/models/wavlm-large and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-st

In [4]:
inputs, rate = torchaudio.load("/home/bltang/Downloads/test/clean.wav")
inputs = torch.cat([inputs, inputs], dim = 0 )
ssl_layers = [1, 3, 7, 12, 18, 23]

In [5]:
toks = discrete_ssl(inputs, SSL_layers = ssl_layers)
output = toks_to_sig(toks[0])

In [6]:
hifi_gan.tokenize = False
wav = hifi_gan(output)

In [10]:
wav.shape

torch.Size([2, 98560])

In [8]:
wav

tensor([[-0.0042, -0.0059, -0.0048,  ...,  0.0108,  0.0101,  0.0244],
        [-0.0042, -0.0059, -0.0048,  ...,  0.0108,  0.0101,  0.0244]])

In [11]:
Audio(wav[0], rate = 16000)